We should call `.py` so that cuda memory will be automatically released after each part.

This notebook is used to organize the codes

## Setup

### What you need to do

- Throw audio files into `data/` directory

- Correctly set `input_list`

- Create a `.env` file in the `process/` directory if using `glm-4`

In [2]:
import os

In [3]:
DATA_PATH = os.getcwd() + '/data/'
MODEL_PATH = '/ssdshare/LLMs/'
MUSIC_PATH = os.getcwd() + '/data/music/'
LLM_MODEL = "glm-4"
GENRATE_MODEL = "playground-v2.5-1024px-aesthetic"
CONTENT_PATH = DATA_PATH + '.tmp/generate/'
STYLE_PATH = DATA_PATH + 'style/illustration_style/'

if not os.path.exists(DATA_PATH + '.tmp/'):
  os.makedirs(DATA_PATH + '.tmp/')

list = ['extract/', 'generate/', 'process/', 'inprompt', 'style_transfer']

for folder in list:
  if not os.path.exists(DATA_PATH + '.tmp/' + folder):
    os.makedirs(DATA_PATH + '.tmp/' + folder)


In [4]:
input_list = [
  'out.mp3',
]
prompts = [r'''
  The name of this song is 晴天. 
''',
]
# Pick the style images in the style library
style_list = [
  # 'opia.png'
]
# You should check both input_list and prompts modified!!!
with open(DATA_PATH + 'input_list.txt', 'w') as f:
  for item in input_list:
    f.write("%s\n" % item)

with open(DATA_PATH + 'style_list.txt', 'w') as f:
  for item in style_list:
    f.write("%s\n" % item)

tmp_list = []
for item in input_list:
  tmp_list.append(item[:-4])
input_list = tmp_list

# if not os.path.exists(DATA_PATH + '.tmp/inprompt/'):
#   os.makedirs(DATA_PATH + '.tmp/inprompt/')
for (prompt, name) in zip(prompts, input_list):
  with open(DATA_PATH + '.tmp/inprompt/' + name + '.prompt', 'w') as f:
    f.write(prompt)

## Extract

In [5]:
os.system(f'python extract/extract.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --music_path {MUSIC_PATH} --output_path {DATA_PATH}.tmp/extract/ --device_num 2')

out.mp3
['out.wav']
audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 9/9 [00:03<00:00,  2.39it/s]


using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
using device 0
successfully add prompt for out.wav
This music is cut into 11 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: This is a song whose genre is Pop, and the lyrics are "可是的故事都讲完了".
Description piece 2: This is a song whose genre is Pop, and the lyrics are "故事的小红帽 从这森林到城堡 谁 给她穿上盛装".
Description piece 3: This is a song whose genre is Pop, and the lyrics are "如果我想要画满世界的大道 为你翘首的那一天 那一天假使那一天 我怎么看不见 早知如此伤眼的阳光 我假装在另一边".
Description piece 4: This is a song whose genre is Pop, and the lyrics are "会不会等到那一天 你才出现 还要多久我才能在你身边 等到风景都看透".
Description piece 5: This is a song whose genre is Pop, and the lyrics are "在你身边的那幅风景的那天 也许我会比较好一点 从前从前 有个人爱你很久 但渐渐风渐渐把距离吹的好远 好不容易 我们再多爱一天".
Description piece 6: This is a song whose genre is Pop, and the lyrics are "

0

In [15]:
for file_name in input_list:
  with open(DATA_PATH + '.tmp/extract/' + file_name + '.prompt', 'r') as f:
    print(f.read())

This music is cut into 11 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: This is a song whose genre is Pop, and the lyrics are "故事的小黄花".
Description piece 2: This is a song whose genre is Pop, and the lyrics are "童年的荡秋千 随记忆一直晃到现在 吹着前奏望着天空".
Description piece 3: This is a song whose genre is Pop, and the lyrics are "我想起花瓣试着掉落 为你翘课的那一天 花落的那一天 教室的那一间 我怎么看不见 消失的下雨天 我好想再淋一遍".
Description piece 4: This is a song whose genre is Pop, and the lyrics are "没想到失去的勇气我还留着 好想再问一遍 你会等待还是离开 刮风这天 我试过握着你手 但偏偏 雨渐渐 大到我看你不见".
Description piece 5: This is a song whose genre is Pop, and the lyrics are "在你身边的那幅风景的那天 也许我会比较好一点 从前从前 有个人爱你很久 但渐渐风渐渐把距离吹的好远 好不容易 我们再多爱一天".
Description piece 6: This is a song whose genre is Pop, and the lyrics are "还要多久 我才能在你身边
等到放晴的那天 也许我会比较好一点 从前从前 有个人爱你很久 但偏偏 风渐渐 把距离吹得好远 好不容易 又能再多爱一天 但故事的最后 你好像还是说了拜拜".
Description piece 7: This is a song whose genre is Pop, and the lyrics are "刮风这天 我试过握着你

## Process

In [23]:
os.system(f'python process/process.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --model {LLM_MODEL} --prompt_path {DATA_PATH}.tmp/extract/ --output_path {DATA_PATH}.tmp/process/')

['out']
Loading model
Model loaded
<class 'zhipuai._client.ZhipuAI'> <class 'NoneType'>
Token spent: 9512


0

In [24]:
for file_name in input_list:
  with open(DATA_PATH + '.tmp/process/' + file_name + '.prompt', 'r') as f:
    print(f.read())
  with open(DATA_PATH + '.tmp/process/' + file_name + '.prompt2', 'rb') as f:
    print(f.read())

sunny day, yellow flowers, Swing set, blowing wind, falling petals, classroom, rainy day, courage, hands held, obscured by rain, waiting, clear skies, distant love, blowing wind, separation, reunion, goodbye, emotional atmosphere, vibrant colors, 8k resolution, 16:9 aspect ratio, 60fps.
b'sunny sky, yellow flowers, childhood swing, memories, falling petals, rainy day, wind, obscured visibility, waiting for clear skies, emotional distance, love story, courage, 8k resolution, 16:9 aspect ratio, 60fps'


## Generate

In [25]:
os.system(f'python generate/generate.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --model {GENRATE_MODEL} --output_path {DATA_PATH}.tmp/generate/ --prompt_path {DATA_PATH}.tmp/process/ --image_num 3')

Loading prompt from file
out.prompt
Prompt loaded
Loading model


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.22it/s]


Model loaded
Generating for out.prompt


100%|██████████| 50/50 [00:17<00:00,  2.92it/s]


Generated for out.prompt
Generating image without characters
Loading prompt from file
Prompt loaded
Generating for out.prompt2


100%|██████████| 50/50 [00:16<00:00,  2.95it/s]


Generated for out.prompt2


0

# Style transfer

In [26]:
os.system(f'python style_transfer/style_transfer.py --data_path {DATA_PATH} --output_path {DATA_PATH}.tmp/style_transfer/ --style_path {STYLE_PATH} --content_path {CONTENT_PATH} -l_o')

['0.png', '1.png', '2.png', '10.png', '11.png', '12.png']
Transferring from /root/LLM_project/codes/data/.tmp/generate/out/0.png to /root/LLM_project/codes/data/style/illustration_style/10.png
Building the style transfer model..


Style Loss : 0.373572 Content Loss: 1.997620:  80%|████████  | 40/50 [00:04<00:01,  9.27it/s]


Transfer from /root/LLM_project/codes/data/.tmp/generate/out/0.png to /root/LLM_project/codes/data/style/illustration_style/10.png done
Transferring from /root/LLM_project/codes/data/.tmp/generate/out/1.png to /root/LLM_project/codes/data/style/illustration_style/10.png
Building the style transfer model..


Style Loss : 0.333863 Content Loss: 2.097287:  80%|████████  | 40/50 [00:04<00:01,  9.37it/s]


Transfer from /root/LLM_project/codes/data/.tmp/generate/out/1.png to /root/LLM_project/codes/data/style/illustration_style/10.png done
Transferring from /root/LLM_project/codes/data/.tmp/generate/out/2.png to /root/LLM_project/codes/data/style/illustration_style/10.png
Building the style transfer model..


...

Style Loss : 0.245253 Content Loss: 0.752173:  80%|████████  | 40/50 [00:04<00:01,  9.37it/s]


Transfer from /root/LLM_project/codes/data/.tmp/generate/out/11.png to /root/LLM_project/codes/data/style/illustration_style/13.png done
Transferring from /root/LLM_project/codes/data/.tmp/generate/out/12.png to /root/LLM_project/codes/data/style/illustration_style/13.png
Building the style transfer model..


Style Loss : 0.214198 Content Loss: 0.695238:  80%|████████  | 40/50 [00:04<00:01,  9.37it/s]


Transfer from /root/LLM_project/codes/data/.tmp/generate/out/12.png to /root/LLM_project/codes/data/style/illustration_style/13.png done


0